In [22]:
from cvrp_parser import parse_cvrplib_uchoa_2014

In [8]:
import os
os.getcwd()

'/Users/constantinosbarmpouris/Documents/MIT/15.C57 Optimization Methods/Project/optimal-vs-heuristics-vrp/CVRP_Heuristic'

In [23]:
# read the file Uchoa_et_al_2014/X-n101-k25.vrp
# NOTE: the VRP files live in the nested 'X/' directory under the dataset folder
file_path = 'Uchoa et al - Vrp-Set-X/X/X-n101-k25.vrp'
#load the text into a string
with open(file_path, 'r') as file:
    vrp_text = file.read()
# parse the CVRP data
inst  = parse_cvrplib_uchoa_2014(vrp_text)

In [10]:
print(inst.name, inst.n_customers, inst.capacity, inst.edge_weight_type)
print("Depot coords:", inst.coords[0])
print("First customer coords:", inst.coords[1])
print("Distance(0,1):", inst.dist(0,1))

X-n101-k25 100 206 EUC_2D
Depot coords: [365. 689.]
First customer coords: [146. 180.]
Distance(0,1): 554.1137067425782


In [12]:
# print the first 5 customers' demands
print("First 5 customers' demands:", inst.demand[1:6])

First 5 customers' demands: [38 51 73 70 58]


In [ ]:
# 

In [15]:
# --- Optimal CVRP (MIP) using Gurobi: single-commodity flow (SCF) formulation
# Requires Gurobi (gurobipy). Set a short TimeLimit for interactive runs.

try:
    import gurobipy as gp
    from gurobipy import GRB
except Exception as e:
    raise ImportError("gurobipy is required to run the MIP. Install/configure Gurobi and its Python API. Original error: {}".format(e))

# Build model from `inst` (parsed above). `inst.coords` has shape (n+1,2); `inst.demand` length n+1; depot is index 0.
n_customers = inst.n_customers
N = list(range(n_customers + 1))  # 0..n where 0 is depot
demand = list(map(int, inst.demand))
capacity = int(inst.capacity)

# Precompute distances (use inst.dist to remain consistent)
dist = [[inst.dist(i, j) for j in N] for i in N]

m = gp.Model("CVRP_SCF")
# Binary arc variables x[i,j] (i->j). We do not allow i==j.
x = m.addVars(N, N, vtype=GRB.BINARY, name='x')
# Continuous flow variables f[i,j] representing load sent on arc i->j (0..capacity).
f = m.addVars(N, N, lb=0.0, ub=capacity, name='f')

# Disable self arcs (force zero)
for i in N:
    m.addConstr(x[i,i] == 0)
    m.addConstr(f[i,i] == 0)

# Each customer must have exactly one incoming and one outgoing arc
for k in range(1, n_customers+1):
    m.addConstr(gp.quicksum(x[k, j] for j in N if j != k) == 1, name=f"out_{k}")
    m.addConstr(gp.quicksum(x[i, k] for i in N if i != k) == 1, name=f"in_{k}")

# Flow conservation: for each customer k, flow in - flow out = demand[k]
for k in range(1, n_customers+1):
    m.addConstr(gp.quicksum(f[i, k] for i in N if i != k) - gp.quicksum(f[k, j] for j in N if j != k) == demand[k], name=f"flow_cons_{k}")

# Link flow and x: f[i,j] <= capacity * x[i,j]
for i in N:
    for j in N:
        if i != j:
            m.addConstr(f[i, j] <= capacity * x[i, j], name=f"cap_link_{i}_{j}")

# Total flow out of depot equals total demand (sum of customer demands)
total_demand = sum(demand[1:])
m.addConstr(gp.quicksum(f[0, j] for j in range(1, n_customers+1)) == total_demand, name='depot_outflow')

# Objective: minimize total distance traveled
m.setObjective(gp.quicksum(dist[i][j] * x[i, j] for i in N for j in N if i != j), GRB.MINIMIZE)

# Tuning: interactive-friendly limits (adjust as needed)
m.Params.TimeLimit = 60  # seconds (set to None or larger for higher-quality solutions)
m.Params.OutputFlag = 1

print('Model: vars=', m.NumVars, 'constrs=', m.NumConstrs)
m.optimize()

if m.Status == GRB.OPTIMAL or m.Status == GRB.TIME_LIMIT or m.Status == GRB.SUBOPTIMAL:
    # Extract chosen arcs
    arcs = [(i, j) for i in N for j in N if i != j and x[i, j].X > 0.5]
    print(f"Solver status: {m.Status}, objective: {m.ObjVal if m.SolCount>0 else None}, arcs: {len(arcs)}")
    # Reconstruct routes from arcs (walk from depot until return)
    succ = {i: j for i, j in arcs}
    routes = []
    used = set()
    for j in range(1, n_customers+1):
        if j in used:
            continue
        # find route start: follow back from j until predecessor is depot or non-existent
        # Build route by starting at depot and following succ links
        route = [0]
        cur = 0
        while True:
            if cur not in succ:
                break
            nxt = succ[cur]
            route.append(nxt)
            used.add(nxt)
            cur = nxt
            if cur == 0:
                break
        routes.append(route)
    # Print routes in readable form (ignore trivial empty ones)
    for r_idx, r in enumerate(routes, start=1):
        # compute route demand and length
        if len(r) <= 1:
            continue
        route_demand = sum(demand[i] for i in r if i != 0)
        route_length = sum(inst.dist(r[i], r[i+1]) for i in range(len(r)-1))
        print(f"Route {r_idx}: {r}  demand={route_demand}  length={route_length:.2f}")
else:
    print('Optimization did not return a solution. Status code:', m.Status)

ImportError: gurobipy is required to run the MIP. Install/configure Gurobi and its Python API. Original error: No module named 'gurobipy'

In [24]:
# Fallback: try Gurobi, otherwise solve the SCF MIP with PuLP (CBC)
# This cell attempts to import gurobipy first. If not available, it will try to use PuLP + CBC.
# If PuLP is not installed, it will attempt to install it via pip (may require internet and permissions).

import sys

try:
    import gurobipy as gp
    from gurobipy import GRB
    print("gurobipy is available — prefer running the earlier Gurobi cell instead of this fallback.")
    # Optionally you could call the Gurobi model-building code here.
except Exception:
    print("gurobipy not available; falling back to PuLP + CBC (open-source)")
    try:
        import pulp
    except Exception:
        print("PuLP not installed — attempting to install via pip. This requires internet access.")
        try:
            import subprocess
            subprocess.check_call([sys.executable, "-m", "pip", "install", "pulp>=2.6.0"])
            import pulp
        except Exception as e:
            raise ImportError("PuLP is required for the fallback and could not be installed automatically. Please run: pip install pulp") from e

    # Build the SCF MIP using PuLP
    # Prepare data from `inst` which should already be defined above
    n_customers = inst.n_customers
    N = list(range(n_customers + 1))
    demand = list(map(int, inst.demand))
    capacity = int(inst.capacity)
    dist = [[inst.dist(i, j) for j in N] for i in N]

    model = pulp.LpProblem('CVRP_SCF_PuLP', pulp.LpMinimize)

    # Variables
    x = pulp.LpVariable.dicts('x', (N, N), lowBound=0, upBound=1, cat=pulp.LpBinary)
    f = pulp.LpVariable.dicts('f', (N, N), lowBound=0, upBound=capacity, cat=pulp.LpContinuous)

    # Objective
    model += pulp.lpSum(dist[i][j] * x[i][j] for i in N for j in N if i != j)

    # Constraints
    # No self arcs
    for i in N:
        model += x[i][i] == 0
        model += f[i][i] == 0

    # Each customer has exactly one incoming and one outgoing arc
    for k in range(1, n_customers + 1):
        model += pulp.lpSum(x[k][j] for j in N if j != k) == 1
        model += pulp.lpSum(x[i][k] for i in N if i != k) == 1

    # Flow conservation: for customers, inflow - outflow = demand[k]
    for k in range(1, n_customers + 1):
        model += pulp.lpSum(f[i][k] for i in N if i != k) - pulp.lpSum(f[k][j] for j in N if j != k) == demand[k]

    # Link flow and x: f[i,j] <= capacity * x[i,j]
    for i in N:
        for j in N:
            if i != j:
                model += f[i][j] <= capacity * x[i][j]

    # Total flow out of depot equals total demand
    total_demand = sum(demand[1:])
    model += pulp.lpSum(f[0][j] for j in range(1, n_customers + 1)) == total_demand

    # Solve with CBC, set time limit (in seconds) for interactive runs
    solver = pulp.PULP_CBC_CMD(timeLimit=60, msg=True)
    print('Model built. Solving with CBC...')
    result = model.solve(solver)

    status = pulp.LpStatus.get(result, None)
    print('Solver status:', status)

    if pulp.LpStatus[result] in ('Optimal', 'Not Solved', 'Integer Feasible') or pulp.LpStatus[result] == 'Optimal':
        arcs = [(i, j) for i in N for j in N if i != j and pulp.value(x[i][j]) is not None and pulp.value(x[i][j]) > 0.5]
        print(f"Arcs selected: {len(arcs)}")
        succ = {i: j for i, j in arcs}
        routes = []
        used = set()
        for j in range(1, n_customers + 1):
            if j in used:
                continue
            route = [0]
            cur = 0
            while True:
                if cur not in succ:
                    break
                nxt = succ[cur]
                route.append(nxt)
                used.add(nxt)
                cur = nxt
                if cur == 0:
                    break
            routes.append(route)
        for r_idx, r in enumerate(routes, start=1):
            if len(r) <= 1:
                continue
            route_demand = sum(demand[i] for i in r if i != 0)
            route_length = sum(inst.dist(r[i], r[i+1]) for i in range(len(r)-1))
            print(f"Route {r_idx}: {r}  demand={route_demand}  length={route_length:.2f}")
    else:
        print('No feasible solution found. Status:', pulp.LpStatus[result])

gurobipy not available; falling back to PuLP + CBC (open-source)
Model built. Solving with CBC...
Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /opt/anaconda3/lib/python3.12/site-packages/pulp/apis/../solverdir/cbc/osx/i64/cbc /var/folders/s7/fy136dw90dsb64z_hj492ptm0000gn/T/3d4a571ace7041959f65fd7bb8886497-pulp.mps -sec 60 -timeMode elapsed -branch -printingOptions all -solution /var/folders/s7/fy136dw90dsb64z_hj492ptm0000gn/T/3d4a571ace7041959f65fd7bb8886497-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 10608 COLUMNS
At line 101613 RHS
At line 112217 BOUNDS
At line 132620 ENDATA
Problem MODEL has 10603 rows, 20402 columns and 60502 elements
Coin0008I MODEL read with 0 errors
seconds was changed from 1e+100 to 60
Option for timeMode changed from cpu to elapsed
Continuous objective value is 24041.9 - 1.53 seconds
Cgl0002I 101 variables fixed
Cgl0004I processed model has 10401 rows, 20200 columns (10100 int

In [17]:
# output the optimal routes and their total length and demand, if a solution was found
def print_routes(routes, demand, inst):
    for r_idx, r in enumerate(routes, start=1):
        if len(r) <= 1:
            continue
        route_demand = sum(demand[i] for i in r if i != 0)
        route_length = sum(inst.dist(r[i], r[i+1]) for i in range(len(r)-1))
        print(f"Route {r_idx}: {r}  demand={route_demand}  length={route_length:.2f}")

print_routes(routes, demand, inst)

Route 1: [0, 97, 91, 51, 0]  demand=203  length=1148.85
Route 2: [0, 97, 91, 51, 0]  demand=203  length=1148.85
Route 3: [0, 97, 91, 51, 0]  demand=203  length=1148.85
Route 4: [0, 97, 91, 51, 0]  demand=203  length=1148.85
Route 5: [0, 97, 91, 51, 0]  demand=203  length=1148.85
Route 6: [0, 97, 91, 51, 0]  demand=203  length=1148.85
Route 7: [0, 97, 91, 51, 0]  demand=203  length=1148.85
Route 8: [0, 97, 91, 51, 0]  demand=203  length=1148.85
Route 9: [0, 97, 91, 51, 0]  demand=203  length=1148.85
Route 10: [0, 97, 91, 51, 0]  demand=203  length=1148.85
Route 11: [0, 97, 91, 51, 0]  demand=203  length=1148.85
Route 12: [0, 97, 91, 51, 0]  demand=203  length=1148.85
Route 13: [0, 97, 91, 51, 0]  demand=203  length=1148.85
Route 14: [0, 97, 91, 51, 0]  demand=203  length=1148.85
Route 15: [0, 97, 91, 51, 0]  demand=203  length=1148.85
Route 16: [0, 97, 91, 51, 0]  demand=203  length=1148.85
Route 17: [0, 97, 91, 51, 0]  demand=203  length=1148.85
Route 18: [0, 97, 91, 51, 0]  demand=203

In [25]:
# Compare the optimal solution to the given solution in the file "/Users/constantinosbarmpouris/Documents/MIT/15.C57 Optimization Methods/Project/optimal-vs-heuristics-vrp/CVRP_Heuristic/Uchoa et al - Vrp-Set-X/X/X-n101-k25.sol"
def parse_solution_file(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()
    routes = []
    for line in lines:  
        line = line.strip()
        if line.startswith("Route"):
            parts = line.split(":")
            if len(parts) > 1:
                route_str = parts[1].strip()
                route = list(map(int, route_str.split()))
                routes.append(route)
    return routes
solution_file_path = 'Uchoa et al - Vrp-Set-X/X/X-n101-k25.sol'
solution_routes = parse_solution_file(solution_file_path)
print("Solution routes from file:")
print_routes(solution_routes, demand, inst)


Solution routes from file:
Route 1: [31, 46, 35]  demand=191  length=246.24
Route 2: [15, 22, 41, 20]  demand=205  length=199.07
Route 3: [1, 70, 54]  demand=201  length=310.57
Route 4: [92, 9, 86]  demand=203  length=166.04
Route 5: [68, 90, 84, 66]  demand=196  length=114.91
Route 6: [76, 55, 16, 69]  demand=203  length=108.88
Route 7: [4, 13, 74]  demand=189  length=269.12
Route 8: [58, 12, 5]  demand=202  length=205.19
Route 9: [18, 10, 39]  demand=206  length=196.51
Route 10: [25, 65, 78, 42, 28]  demand=205  length=220.37
Route 11: [7, 2, 45, 43, 29, 36, 72, 57]  demand=206  length=808.82
Route 12: [87, 37, 6, 49, 14]  demand=206  length=631.76
Route 13: [3, 77, 63]  demand=202  length=163.56
Route 14: [44, 67, 88, 40]  demand=202  length=222.92
Route 15: [82, 60, 59]  demand=205  length=81.58
Route 16: [8, 17]  demand=172  length=56.08
Route 17: [34, 64, 96, 48, 26, 47, 38]  demand=202  length=597.61
Route 18: [80, 94, 56, 21]  demand=204  length=311.34
Route 19: [71, 62, 99, 98

In [26]:
# Compare the 2 solutions by computing total length and demand for each route, and the overall total length and demand. Print the results in a readable format.
def compute_solution_stats(routes, demand, inst):
    total_length = 0.0
    total_demand = 0
    for r_idx, r in enumerate(routes, start=1):
        if len(r) <= 1:
            continue
        route_demand = sum(demand[i] for i in r if i != 0)
        route_length = sum(inst.dist(r[i], r[i+1]) for i in range(len(r)-1))
        print(f"Route {r_idx}: {r}  demand={route_demand}  length={route_length:.2f}")
        total_length += route_length
        total_demand += route_demand
    print(f"Total demand: {total_demand}, Total length: {total_length:.2f}")
print("Optimal solution stats:")
compute_solution_stats(routes, demand, inst)
print("Given solution stats:")
compute_solution_stats(solution_routes, demand, inst)

Optimal solution stats:
Route 1: [0, 97, 91, 51, 0]  demand=203  length=1148.85
Route 2: [0, 97, 91, 51, 0]  demand=203  length=1148.85
Route 3: [0, 97, 91, 51, 0]  demand=203  length=1148.85
Route 4: [0, 97, 91, 51, 0]  demand=203  length=1148.85
Route 5: [0, 97, 91, 51, 0]  demand=203  length=1148.85
Route 6: [0, 97, 91, 51, 0]  demand=203  length=1148.85
Route 7: [0, 97, 91, 51, 0]  demand=203  length=1148.85
Route 8: [0, 97, 91, 51, 0]  demand=203  length=1148.85
Route 9: [0, 97, 91, 51, 0]  demand=203  length=1148.85
Route 10: [0, 97, 91, 51, 0]  demand=203  length=1148.85
Route 11: [0, 97, 91, 51, 0]  demand=203  length=1148.85
Route 12: [0, 97, 91, 51, 0]  demand=203  length=1148.85
Route 13: [0, 97, 91, 51, 0]  demand=203  length=1148.85
Route 14: [0, 97, 91, 51, 0]  demand=203  length=1148.85
Route 15: [0, 97, 91, 51, 0]  demand=203  length=1148.85
Route 16: [0, 97, 91, 51, 0]  demand=203  length=1148.85
Route 17: [0, 97, 91, 51, 0]  demand=203  length=1148.85
Route 18: [0, 97

In [27]:
def check_feasibility(routes, demand, capacity):
    n = len(demand)-1
    seen = []
    for r in routes:
        # Expect route either like [0, a, b, ..., 0] or [a,b,...] depending on format
        # Normalize: remove depot markers if present
        nodes = [v for v in r if v != 0]
        seen.extend(nodes)
        rd = sum(demand[v] for v in nodes)
        if rd > capacity:
            return False, f"Route {r} exceeds capacity ({rd} > {capacity})"
        for v in nodes:
            if v < 1 or v > n:
                return False, f"Invalid node index {v} in route {r}"
    # check coverage
    from collections import Counter
    cnt = Counter(seen)
    missing = [i for i in range(1, n+1) if cnt[i] == 0]
    dup = [i for i, c in cnt.items() if c > 1]
    if missing:
        return False, f"Missing customers: {missing[:10]}{'...' if len(missing)>10 else ''}"
    if dup:
        return False, f"Duplicate visits: {dup[:10]}{'...' if len(dup)>10 else ''}"
    return True, "Feasible (coverage and capacity OK)"

# Example usage:
ok, msg = check_feasibility(solution_routes, demand, capacity)
print(ok, msg)

True Feasible (coverage and capacity OK)
